# Overnight News Sentiment
With some scraping magic, our team collected news headlines for our `FANG` Stocks from `https://www.nasdaq.com/symbol/{ticker}/news-headlines` into `dataset/nasdaq/FANG.csv`

With the `mkt_dt_utils` and `sentiment_utils` we created, we can compute **Overnight News Sentiments** for a given stock for the purpose of doing studies against overnight stock returns

## Let's Load Our Data

In [15]:
import pandas as pd
import numpy as np

In [16]:
fname = 'dataset/nasdaq/raw.csv'
data = pd.read_csv( fname, header = 0, index_col = None)

In [17]:
data.tail()

,datetime,stockcode,source,headline,article,urls
6400,"February 04, 2019, 03:26:00 PM EDT",NFLX,Zacks.com,"Super Bowl Spending, Streaming & Amazon's Big ...",\n Welcome to the latest episode of the Fu...,https://www.nasdaq.com/article/super-bowl-spen...
6401,"February 04, 2019, 01:24:00 PM EDT",NFLX,Zacks.com,Buy Apple (AAPL) Stock on Streaming TV Push to...,\n Shares of Apple AAPL jumped over 2.5% Mond...,https://www.nasdaq.com/article/buy-apple-aapl-...
6402,"February 04, 2019, 12:42:43 PM EDT",NFLX,InvestorPlace Media,3 Must-Watch Comic Book Movies on Netflix,"\nInvestorPlace - Stock Market News, Stock Adv...",https://www.nasdaq.com/article/3-must-watch-co...
6403,"February 04, 2019, 12:36:05 PM EDT",NFLX,InvestorPlace Media,2 Crazy Acquisitions That Would Boost Alphabet...,"\nInvestorPlace - Stock Market News, Stock Adv...",https://www.nasdaq.com/article/2-crazy-acquisi...
6404,"February 04, 2019, 04:38:00 PM EDT",NFLX,NaN,US STOCKS-Boost in tech shares sends Wall Stre...,\n\n\nShutterstock photo\n\n@media screen and ...,https://www.nasdaq.com/article/us-stocksboost-...


In [18]:
# data['timestring'] = data.index
# data['timestring'] = data['timestring'].astype('str')
data['datetime'] = data['datetime'].astype('str')
data['tz'] = data['datetime'].apply( lambda x: x[-3:])

In [19]:
data[data['tz']!= 'EDT']

,datetime,stockcode,source,headline,article,urls,tz
545,nan,AMZN,NaN,"The Zacks Analyst Blog Highlights: Netflix, Di...",\n\n\nShutterstock photo\n\n\n\n\n\n\n\r\n ...,https://www.nasdaq.com/article/the-zacks-analy...,nan
551,nan,AMZN,NaN,Amazon Announces New Job Positions for New Yor...,\n\n\nShutterstock photo\n\n\n\n\n\n\n\r\n ...,https://www.nasdaq.com/article/amazon-announce...,nan
582,nan,AMZN,NaN,Why Amazon (AMZN) is Poised to Beat Earnings E...,\n\n\nShutterstock photo\n\n\n\n\n\n\n\r\n ...,https://www.nasdaq.com/article/why-amazon-amzn...,nan
602,nan,AMZN,NaN,Amazon (AMZN) Dips More Than Broader Markets: ...,\n\n\nShutterstock photo\n\n\n\n\n\n\n\r\n ...,https://www.nasdaq.com/article/amazon-amzn-dip...,nan
2061,nan,GOOGL,NaN,Amazon Announces New Job Positions for New Yor...,\n\n\nShutterstock photo\n\n\n\n\n\n\n\r\n ...,https://www.nasdaq.com/article/amazon-announce...,nan
2089,nan,GOOGL,NaN,Alphabet (GOOGL) Dips More Than Broader Market...,\n\n\nShutterstock photo\n\n\n\n\n\n\n\r\n ...,https://www.nasdaq.com/article/alphabet-googl-...,nan
4992,nan,NFLX,NaN,"The Zacks Analyst Blog Highlights: Netflix, Di...",\n\n\nShutterstock photo\n\n\n\n\n\n\n\r\n ...,https://www.nasdaq.com/article/the-zacks-analy...,nan


### Remove Bad Data

In [29]:
dup_filter = data.duplicated()
na_dt = pd.to_datetime(data['datetime']).isna() #data.datetime.isna()

print(f'Dataset has {len(data)} points, of which; {sum(dup_filter)} are duplicates, and {sum(na_dt)} are missing DateTime info.')

Dataset has 6021 points, of which; 0 are duplicates, and 7 are missing DateTime info.


In [30]:
data = data.drop_duplicates()
data = data[ - pd.to_datetime(data.datetime).isna()]
print(f'After cleaning we have {len(data)} data points')

/Users/JHO/.virtualenvs/VADER/lib/python3.6/site-packages/dateutil/parser/_parser.py:1206: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


After cleaning we have 6014 data points


In [22]:
data['tz'] = data['tz'].apply( lambda x : 'EST' if x == 'EDT' else x)
data['dt'] = pd.to_datetime(data['datetime']).dt.tz_localize('EST')

In [8]:
data.head()

,datetime,stockcode,source,headline,article,urls,tz,dt
0,"February 07, 2019, 09:11:00 PM EDT",AMZN,NaN,Amazon CEO Jeff Bezos Accuses National Enquire...,\n\n\nShutterstock photo\n\n@media screen and ...,https://www.nasdaq.com/article/amazon-ceo-jeff...,EST,2019-02-07 21:11:00-05:00
1,"February 07, 2019, 02:49:22 PM EDT",AMZN,BNK Invest,"Notable Thursday Option Activity: AMZN, BKNG, ...",\nAmong the underlying components of the S&P 5...,https://www.nasdaq.com/article/notable-thursda...,EST,2019-02-07 14:49:22-05:00
2,"February 07, 2019, 06:55:00 PM EDT",AMZN,NaN,Amazon's Bezos says National Enquirer owner tr...,\n\n\nReuters\n\n@media screen and (max-device...,https://www.nasdaq.com/article/amazons-bezos-s...,EST,2019-02-07 18:55:00-05:00
3,"February 07, 2019, 06:26:00 PM EDT",AMZN,NaN,Amazon's Bezos says National Enquirer tried to...,\n\n\nReuters\n\n@media screen and (max-device...,https://www.nasdaq.com/article/amazons-bezos-s...,EST,2019-02-07 18:26:00-05:00
4,"February 07, 2019, 11:55:45 AM EDT",AMZN,InvestorPlace Media,"IRS Tax Refund 2019: So, Where’s My Tax Refund?","\nInvestorPlace - Stock Market News, Stock Adv...",https://www.nasdaq.com/article/irs-tax-refund-...,EST,2019-02-07 11:55:45-05:00


## Read Pre-existing DF and only work on New records

In [128]:
output_fname = 'dataset/nasdaq/overnight_sentiments.csv'

old_df = pd.read_csv(output_fname, index_col = 0)

l_new = [(url in old_df['urls']) for url in data['urls']]

sum(l_new)

0

In [132]:
test_url = data['urls'][0]
test_url

'https://www.nasdaq.com/article/amazon-ceo-jeff-bezos-accuses-national-enquirer-owner-of-extortion--blackmail-20190207-01538'

In [133]:
test_url in old_df['urls']

False

In [130]:
old_df['urls'][0]

'https://www.nasdaq.com/article/amazon-ceo-jeff-bezos-accuses-national-enquirer-owner-of-extortion--blackmail-20190207-01538'

## Label Overnight News and Output Next Market Date

In [38]:
from datetime import datetime
from mkt_dt_utils import IsMarketOpen, GetNextMktDate, days_hours_mins_secs

temp_fname = 'dataset/nasdaq/overnight_sentiments_temp.csv'
ExchgName = 'NYSE'
data_limit = None #10

df = data.drop(columns= ['tz'])
if data_limit:
    df = df[: data_limit]

stime = datetime.now()
print(f'Filtering for {len(df)} Overnight News Articles...')
df['IsMarketOpen'] = df['dt'].apply(
    lambda x: IsMarketOpen(x.to_pydatetime(), ExchgName)    
    )

df.to_csv(temp_fname)

print(f'Determining Trade Date for Overnight News Articles...')
df_on = df[df['IsMarketOpen'] == False]
df_on['TradeDate'] = df_on.apply(
    lambda x: GetNextMktDate(x['dt'].to_pydatetime(),ExchgName),
    axis =1
    )

df_on.to_csv(temp_fname)

ttime = datetime.now() - stime
d_ , h_, m_ , s_ = days_hours_mins_secs(ttime)
print(f'Time elapsed {h_} hours, {m_} minutes, {s_} seconds.')

Filtering for 6014 Overnight News Articles...
Determining Trade Date for Overnight News Articles...


/Users/JHO/.virtualenvs/VADER/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Time elapsed 1 hours, 29 minutes, 47 seconds.


In [39]:
print(f'--- Only {len(df_on)} of {len(df)} are overnight news ---')

--- Only 3148 of 6014 are overnight news ---


## Generate Relevance + Sentiment for each Article

In [17]:
import pandas as pd
from datetime import datetime
from mkt_dt_utils import days_hours_mins_secs

temp_fname = 'dataset/nasdaq/overnight_sentiments_temp.csv'

#df_ = df_on
tmp_fname = 'dataset/nasdaq/overnight_sentiments_temp_0.csv'
df_ = pd.read_csv(tmp_fname, index_col = 0)

In [18]:
from sentiment_utils import GetCleanText, GetSummary, GetRelavency, GetSentimentScore, Stock_KW_Dict

stime = datetime.now()

# 1. Clean + Summarize
print(f'Summarizing {len(df_)} overnight news article...')
df_['summary'] = df_['article'].apply(
    lambda x : GetSummary(GetCleanText( x ), method = 'sumy-lex_rank')
)

df_.to_csv(temp_fname)

ttime = datetime.now() - stime
d_ , h_, m_ , s_ = days_hours_mins_secs(ttime)
print(f'Time elapsed {h_} hours, {m_} minutes, {s_} seconds.')

Summarizing 3148 overnight news article...
Time elapsed 0 hours, 3 minutes, 29 seconds.


In [17]:
# 3. Relavency
print(f'Determining {len(df_)} overnight news article relavance...')
df_['_relevance'] = df_.apply(
    lambda x: GetRelavency( x['summary'], Stock_KW_Dict[ x['stockcode']], debugmode = False),
    axis = 1
)

df_.to_csv(temp_fname)

Determining overnight news article relavance...


/Users/JHO/.virtualenvs/VADER/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [19]:
# 4. Sentiment
print(f'Generating Sentiment Score...')
df_on['_sentiment'] = df_on['summary'].apply(
    lambda x: GetSentimentScore( x , method = 'bespoke') if x != '' else np.nan
)

df_on.to_csv(temp_fname)

Generating Sentiment Score...


/Users/JHO/.virtualenvs/VADER/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [21]:
df_on.head()

,datetime,stockcode,source,headline,article,urls,dt,IsMarketOpen,TradeDate,summary,_relevance,_sentiment
0,"February 07, 2019, 09:11:00 PM EDT",AMZN,NaN,Amazon CEO Jeff Bezos Accuses National Enquire...,\n\n\nShutterstock photo\n\n@media screen and ...,https://www.nasdaq.com/article/amazon-ceo-jeff...,2019-02-07 21:11:00-05:00,False,2019-02-08,Shutterstock photo@media screen and (Amazon CE...,"{'Count': 11, 'sent_count': 3, 'freq': 3.66666...",0.717833
2,"February 07, 2019, 06:55:00 PM EDT",AMZN,NaN,Amazon's Bezos says National Enquirer owner tr...,\n\n\nReuters\n\n@media screen and (max-device...,https://www.nasdaq.com/article/amazons-bezos-s...,2019-02-07 18:55:00-05:00,False,2019-02-08,"Jeff Bezos, chief executive of Amazon.com Inc,...","{'Count': 4, 'sent_count': 5, 'freq': 0.8}",0.647160
3,"February 07, 2019, 06:26:00 PM EDT",AMZN,NaN,Amazon's Bezos says National Enquirer tried to...,\n\n\nReuters\n\n@media screen and (max-device...,https://www.nasdaq.com/article/amazons-bezos-s...,2019-02-07 18:26:00-05:00,False,2019-02-08,"Jeff Bezos, chief executive of Amazon.com Inc,...","{'Count': 8, 'sent_count': 5, 'freq': 1.6}",0.577320
22,"February 07, 2019, 09:25:09 AM EDT",AMZN,Motley Fool,Only 1 Amazon Metric Should Really Matter to I...,\n Wall Street didn't show much love for Ama...,https://www.nasdaq.com/article/only-1-amazon-m...,2019-02-07 09:25:09-05:00,False,2019-02-07,Wall Street didn't show much love for Amazon...,"{'Count': 5, 'sent_count': 5, 'freq': 1.0}",0.388820
23,"February 07, 2019, 08:36:00 AM EDT",AMZN,Motley Fool,Amazon Is Running Away With the Smart-Speaker ...,\n The latest data for the smart-speaker marke...,https://www.nasdaq.com/article/amazon-is-runni...,2019-02-07 08:36:00-05:00,False,2019-02-07,That's because Amazon makes it easy for user...,"{'Count': 7, 'sent_count': 5, 'freq': 1.4}",0.475180


In [42]:
from sentiment_utils import FilterOutCssJs
text = df_on['article'][0]
tt = FilterOutCssJs(GetCleanText( text ))
tt

'Shutterstock photo@media screen and (Amazon CEO Jeff Bezos Accuses National Enquirer Owner Of \'extortion & Blackmail\' (RTTNews.com) - Jeff Bezos, owner of the Washington Post and founder of Amazon.com Inc, has accused the National Enquirer\'s owner of trying to blackmail him over lewd pictures.Bezos said the magazine\'s parent company American Media Inc or AMI wanted him to drop an investigation into how it obtained his private messages.Bezos and his wife, MacKenzie, said last month they were getting divorced.Their announcement came just before the National Enquirer carried reports about the tech titan\'s extramarital affair.In a blog post on Thursday, Bezos posted an email he said had been sent to his intermediaries by AMI\'s representatives threatening to publish "intimate photos" of Bezos and his lover, former TV host Lauren Sanchez.The billionaire said AMI had wanted him to make a "false public statement" that the National Enquirer\'s coverage of him and his mistress was not pol

In [37]:
summ = GetSummary(tt, method = 'sumy')
summ

(<Sentence: One of the biggest sellers on Amazon.com Inc'sIndia website has returned after the online retail giant changed its business structures to comply with new federal e-commerce curbs that took effect last week, two sources told Reuters.India's modified foreign direct investment rules that kicked in on Feb. 1, prevent companies such as Amazon from selling products from vendors where they, or their group companies, have equity holdings.>,
 <Sentence: But on Thursday Cloudtail had returned with more than 300,000 products listed on Amazon, after the U.S. e-commerce firm cut its indirect holding to 24 percent, one of the sources with knowledge of the matter told Reuters.>,
 <Sentence: The stake was bought by the majority holders, Catamaran Ventures, the source said.>,
 <Sentence: "We have no equity participation in any seller company on our marketplace," Amazon said in a statement, without sharing details of the changes in its indirect holding of Cloudtail.Catamaran, an investment f

In [39]:
len( summ)

5

In [41]:
" ".join([str(sent) for sent in summ])

'One of the biggest sellers on Amazon.com Inc\'sIndia website has returned after the online retail giant changed its business structures to comply with new federal e-commerce curbs that took effect last week, two sources told Reuters.India\'s modified foreign direct investment rules that kicked in on Feb. 1, prevent companies such as Amazon from selling products from vendors where they, or their group companies, have equity holdings. But on Thursday Cloudtail had returned with more than 300,000 products listed on Amazon, after the U.S. e-commerce firm cut its indirect holding to 24 percent, one of the sources with knowledge of the matter told Reuters. The stake was bought by the majority holders, Catamaran Ventures, the source said. "We have no equity participation in any seller company on our marketplace," Amazon said in a statement, without sharing details of the changes in its indirect holding of Cloudtail.Catamaran, an investment firm launched by Infosys  founder N.R. Narayana Murt

In [106]:
ddf = df_on[df_on['summary']=='']
ddf = ddf.reset_index()
ddf.loc[3,['article']].values

array(['\n\n\nReuters\n\n@media screen and (max-device-width: 480px){body{ font-size:30pt !important;}}\n\n\n\nBy Aditya Kalra and Sankalp Phartiyal\nNEW DELHI, Feb 7 (Reuters) - One of the biggest sellers on Amazon.com Inc\'sIndia website has returned after the online retail giant changed its business structures to comply with new federal e-commerce curbs that took effect last week, two sources told Reuters.\nIndia\'s modified foreign direct investment rules that kicked in on Feb. 1, prevent companies such as Amazon from selling products from vendors where they, or their group companies, have equity holdings. makeAd(\'4\',\'300x250\',\'mktsnews\',\'article\',\'\',\'\');\nBut on Thursday Cloudtail had returned with more than 300,000 products listed on Amazon, after the U.S. e-commerce firm cut its indirect holding to 24 percent, one of the sources with knowledge of the matter told Reuters. The stake was bought by the majority holders, Catamaran Ventures, the source said.\n"We have no e

In [108]:
tt = ddf.loc[3,['article']].values[0]
GetCleanText(tt)

'Reuters@media screen and (max-device-width: 480px){body{ font-size:30pt !important;}}By Aditya Kalra and Sankalp PhartiyalNEW DELHI, Feb 7 (Reuters) - One of the biggest sellers on Amazon.com Inc\'sIndia website has returned after the online retail giant changed its business structures to comply with new federal e-commerce curbs that took effect last week, two sources told Reuters.India\'s modified foreign direct investment rules that kicked in on Feb. 1, prevent companies such as Amazon from selling products from vendors where they, or their group companies, have equity holdings. makeAd(\'4\',\'300x250\',\'mktsnews\',\'article\',,);But on Thursday Cloudtail had returned with more than 300,000 products listed on Amazon, after the U.S. e-commerce firm cut its indirect holding to 24 percent, one of the sources with knowledge of the matter told Reuters. The stake was bought by the majority holders, Catamaran Ventures, the source said."We have no equity participation in any seller company

In [122]:
data.columns

Index(['datetime', 'stockcode', 'source', 'headline', 'article', 'urls', 'tz',
       'dt'],
      dtype='object')

In [114]:
df_on.head()

,datetime,stockcode,source,headline,article,urls,dt,IsMarketOpen,TradeDate,summary,_relevance,_sentiment
0,"February 07, 2019, 09:11:00 PM EDT",AMZN,NaN,Amazon CEO Jeff Bezos Accuses National Enquire...,\n\n\nShutterstock photo\n\n@media screen and ...,https://www.nasdaq.com/article/amazon-ceo-jeff...,2019-02-07 21:11:00-05:00,False,2019-02-08,,0,NaN
2,"February 07, 2019, 06:55:00 PM EDT",AMZN,NaN,Amazon's Bezos says National Enquirer owner tr...,\n\n\nReuters\n\n@media screen and (max-device...,https://www.nasdaq.com/article/amazons-bezos-s...,2019-02-07 18:55:00-05:00,False,2019-02-08,Reuters@media screen and (max-device-width: 48...,2,0.452660
3,"February 07, 2019, 06:26:00 PM EDT",AMZN,NaN,Amazon's Bezos says National Enquirer tried to...,\n\n\nReuters\n\n@media screen and (max-device...,https://www.nasdaq.com/article/amazons-bezos-s...,2019-02-07 18:26:00-05:00,False,2019-02-08,"In it, AMI proposes a public acknowledgment fr...",0,0.329825
22,"February 07, 2019, 09:25:09 AM EDT",AMZN,Motley Fool,Only 1 Amazon Metric Should Really Matter to I...,\n Wall Street didn't show much love for Ama...,https://www.nasdaq.com/article/only-1-amazon-m...,2019-02-07 09:25:09-05:00,False,2019-02-07,But as Motley Fool Money host Chris Hil...,8,0.332850
23,"February 07, 2019, 08:36:00 AM EDT",AMZN,Motley Fool,Amazon Is Running Away With the Smart-Speaker ...,\n The latest data for the smart-speaker marke...,https://www.nasdaq.com/article/amazon-is-runni...,2019-02-07 08:36:00-05:00,False,2019-02-07,"Aside from Amazon's dominance, the smart-speak...",9,0.451383


In [117]:
df_on.describe()

,_relevance,_sentiment
count,259.000000,233.000000
mean,2.000000,0.398067
std,3.441651,0.323818
min,0.000000,-0.785500
25%,0.000000,0.285456
50%,0.000000,0.440580
75%,3.000000,0.579443
max,19.000000,0.973000


### Add new data into CSV file

In [118]:
df_all = df_on

## Write DF to File

In [119]:
l_req_col = ['datetime', 'stockcode', 'source', 'headline', 'article', 'urls']
output_fname = 'dataset/nasdaq/overnight_sentiments.csv'

df_all.to_csv(output_fname)